In [1]:
%reload_ext autoreload
%autoreload 2

from pprint import pprint
import csv
from copy import deepcopy
import json
import pathlib
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import src.annotation_utils as a_utils
import src.llm_utils as llm_utils
import src.message_utils as m_utils
from src.env import (
    BRAT_DATA_PATH,
)
import os

load_dotenv()


## Load evaluation dataset

### Load from a previous fine-tune job

In [ ]:
job_desc_dir = None

training_set, validation_set, test_set, fine_tuned_model_id = llm_utils.load_eval_info(job_desc_dir)

len(training_set), len(validation_set), len(test_set)

### Or, Load a fresh dataset

In [10]:
## For (segment, data_span)
# data_entities = a_utils.load_data_entities_of_segments()

# data_entities = [segment for segment in data_entities if segment['entities']]

# For data span
# all_data = m_utils.as_training_data_for_data_span_of_segment(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_segment_1_1(data_entities)
# For data classification
# all_data = m_utils.as_training_data_for_data_classification_of_segment(data_entities)
# For data classification (gradual, level 0)
# all_data = m_utils.as_training_data_for_data_classification_of_segment_gradual(data_entities)

## For (segment, sentence, data_span)
# data_entities = a_utils.load_data_entities_of_sentences()
# For data span of sentence
# all_data = m_utils.as_training_data_for_data_span_of_sentence(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_sentence_1(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_sentence_only(data_entities)


## For (sentence, purpose_span)
purpose_entities = a_utils.load_purpose_entities_of_sentences()
# For purpose span of sentence
all_data = m_utils.as_training_data_for_purpose_span_of_sentence_only(purpose_entities)


test_set = all_data

len(test_set), test_set[0]

(1067,
 {'messages': [{'role': 'system',
    'content': 'You are an annotation expert. You will be given a segment of a privacy policy of a web or mobile application, and will be asked to annotate purpose entities in it.\n\nIMPORTANT: Filtering Out General Phrase\nBefore annotating, carefully check each potential purpose entity. DO NOT annotate general phrases that do not provide specific purpose types.\nExamples of general phrases to omit include, but are not limited to:\n\n"other purposes"\n"purposes described in our policy"\n\nPurpose entities are phrases in segment text that refer to the purposes for which USER\'S PERSONAL DATA will be used, collected, processed, protected, shared with the third parties, etc. The purpose entity must be mentioned in one of the following context types:\n1. first-party-collection-use - the policy segment mentions collection, usage, processing, storage, retention, deletion, or protection of personal user datum or data by the first party (the applicatio

## Run model evaluation

In [8]:
model_id = fine_tuned_model_id if 'fine_tuned_model_id' in locals() else 'gpt-4o-mini-2024-07-18'
# model_id = '4.0Ultra'
model_id = 'gpt-4o-2024-08-06'
model_id

'gpt-4o-2024-08-06'

In [9]:
messages_list = [data['messages'][:-1] for data in test_set]
correct_outputs = [data['messages'][-1]['content'] for data in test_set]

dir_name, obj_model_outputs = llm_utils.query_llm(model_id, messages_list, correct_outputs=correct_outputs,
                                                  batch=False,
                                                  desc='data_span-sent_entity')
dir_name, len(obj_model_outputs)

# Not using batch for some tasks because of rate limit
# dir_name, batch_job = llm_utils.query_llm(model_id, messages_list, correct_outputs=correct_outputs,
#                                                   batch=True,
#                                                   desc='data_span-seg_entity-ver2')
# dir_name, batch_job

100%|██████████| 867/867 [14:19<00:00,  1.01it/s]


('eval-2024-09-17-22-35-35-gpt-4o-2024-08-06', 867)

In [37]:
llm_utils.wait_for_batch_job_finish(batch_job.id)
# llm_utils.retrieve_batch_query_result()
# llm_utils.combine_batch_query_result()

Batch(id='batch_JqFQSt6gdH8BBa7BHgVYpeFH', completion_window='24h', created_at=1726578398, endpoint='/v1/chat/completions', input_file_id='file-bWlE4mEreTE1ns9Va1l1Gr69', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-2024-08-06 in organization org-B2C2pNzAq4paAOvhIYdFJlSv. Limit: 90,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1726664798, failed_at=1726578399, finalizing_at=None, in_progress_at=None, metadata={'description': 'data_span-seg_entity-ver2'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))